In [1]:
from src.c_country import C_Country
from utils.hypergraph_generator import get_hypergraph


import csv
import json
import numpy as np
import pandas as pd
import networkx as nx
import random as rnd

import seaborn as sn
import matplotlib.pyplot as plt
sn.set_theme(style="whitegrid")

import multiprocessing
from subprocess import Popen, STDOUT, PIPE
import copy
import time

In [10]:
start = time.time()
H=get_hypergraph('barabasi', args={'n' : 5000, 'm':1, 'distribution': 'uniform','size':4})
print('running time: ' + str(time.time() - start))

running time: 37.904765367507935


In [11]:
betas=[0.01,0.05,0.1,0.15,0.2]

In [12]:
args = {
    "--beta": 0.2,
    "--seed": 0,
    "--sigma": 1.0, 
    "--mu": 0.2,
    "--max_sim": 1000,
    "inf_agent_num":5,
}
args["procnum"] = 10
args["simnum"] = 5

In [13]:
h=C_Country(H)
node_num = h.node_num

In [14]:
nodes = [node for node in H.nodes]

In [15]:
start = time.time()
res = {}
pool = multiprocessing.Pool(processes=args["procnum"])
manager = multiprocessing.Manager()
lock = manager.Lock()
str_h=h.get_str_hypergraph()

job_count = manager.Array("i", [0,len(betas)*args["simnum"]])
for beta in betas:
    res[beta]=[]
    for i in range(args["simnum"]):
        act_args = copy.copy(args)
        act_args["--seed"]=i
        act_args["--beta"]=beta
        inf_nodes = np.random.choice(nodes, size = args['inf_agent_num'], replace = False)
        history1 = pool.apply_async(C_Country.run, args =
                       (act_args, str_h, node_num, job_count, lock,
                        inf_nodes))
        #history1 = hun.run(args, inf_area)
        res[beta].append(history1)
pool.close()
pool.join()
#print('') # After logging
for beta in betas:
    res[beta] = [t1.get() for t1 in res[beta]]
print(' running time: ' + str(time.time() - start))

 25/25running time: 25.975918292999268
